In [1]:
import torch
from torch.jit import script, trace
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import csv
import random
import os
import unicodedata
import re
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')
print(device)

cuda


In [2]:
corpus_name = 'cornell movie-dialogs corpus'
corpus = os.path.join('data', corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as dataFile:
        lines = dataFile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [3]:
# split each line of file into a dictionary of fields
def loadLines(filename, fields):
    lines = {}
    with open(filename, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines

# groups fields of lines from 'loadLines' into conversations based on *movie_conversations.txt*
def loadConversations(filename, lines, fields):
    conversations = []
    with open(filename, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            lineIds = eval(convObj['utteranceIDs'])
            convObj['lines'] = []
            for lineId in lineIds:
                convObj['lines'].append(lines[lineId])
            conversations.append(convObj)
    return conversations

# extract pair of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        for i in range(len(conversation['lines']) - 1):
            inputLine = conversation['lines'][i]['text'].strip()
            targetLine = conversation['lines'][i+1]['text'].strip()
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [4]:
# define path to new file
datafile = os.path.join(corpus, 'formatted_movie_lines.txt')

delimiter = '\t'
# unescape the delimiter
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# initialize the line dictionary, conversations list, field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ['lineID', 'characterID', 'movieID', 'character', 'text']
MOVIE_CONVERSATIONS_FIELDS = ['character1ID', 'character2ID', 'movieID', 'utteranceIDs']

print('\nProcesing corpus...')
lines = loadLines(os.path.join(corpus, 'movie_lines.txt'), MOVIE_LINES_FIELDS)

print('\nLoading Conversations...')
conversations = loadConversations(os.path.join(corpus, 'movie_conversations.txt'), lines, MOVIE_CONVERSATIONS_FIELDS)

# write new csv files
print('\nWriting newly formatted file...')
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

print('\nSample lines from file: ')
printLines(datafile)


Procesing corpus...

Loading Conversations...

Writing newly formatted file...

Sample lines from file: 
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

In [5]:
# Creating vocabulary

PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #end of sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: 'PAD', SOS_token: 'SOS', EOS_token:'EOS'}
        self.num_words = 3 #count sos, eos, pad
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
    
#     Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        
        print('keep_words {} / {} = {:.4f}'.format(
        len(keep_words), len(self.word2index), len(keep_words)/len(self.word2index)))
        
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: 'PAD', SOS_token:'SOS', EOS_token: 'EOS'}
        self.num_words = 3
        
        for word in keep_words:
            self.addWord(word)

In [6]:
MAX_LENGTH = 10 # maximux sentence length to consider

# turn a unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# lowercase, trim and remove non letter character
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print('Reading Lines...')
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# returns true iff both sentences in a pair 'p' are under MAX_LENGTH threshold
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair()
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print('Start preparing training data...')
    voc, pairs = readVocs(datafile, corpus_name)
    print('Read {!s} sentence pairs'.format(len(pairs)))
    pairs = filterPairs(pairs)
    print('Trimmed to {!s} sentence pairs'.format(len(pairs)))
    print('Counting words...')
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print('Counted words: ', voc.num_words)
    return voc, pairs

# load/assemble voc ad pairs
save_dir = os.path.join('data', 'save')
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# print some pairs to validate
print('\nPairs: ')
for pair in pairs[:10]:
    print(pair)

Start preparing training data...
Reading Lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words:  18008

Pairs: 
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [7]:
MIN_COUNT = 3 # min word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
        
        if keep_input and keep_output:
#             print(pair)
            keep_pairs.append(pair)
    
    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    
    return keep_pairs

pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


In [38]:
def indexFromSentences(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue = PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexFromSentences(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, max target length
def outputVar(l, voc):
    indexes_batch = [indexFromSentences(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(' ')), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variables, lengths, target_variables, mask, max_target_len = batches
print('Input variables: ', input_variables)
print('Lengths: ', lengths)
print('Target variables: ', target_variables)
print('mask: ', mask)
print('max_target_length: ', max_target_len)

Input variables:  tensor([[ 360,    5,  582,   25,  659],
        [ 227,   92,  151,   18,    6],
        [ 784,    7,   53, 1114,    2],
        [  83,  123, 6922,    4,    0],
        [ 403,    6,   66,    2,    0],
        [   4,    2,    2,    0,    0],
        [   2,    0,    0,    0,    0]])
Lengths:  tensor([7, 6, 6, 5, 3])
Target variables:  tensor([[  62,   90,  582,  159,  201],
        [   4,    4,  151,  101,  164],
        [   2,    4,   53, 1381,    4],
        [   0,    4, 6922,    4,  287],
        [   0,    2,   66,    2,  577],
        [   0,    0,    2,    0,  382],
        [   0,    0,    0,    0,  154],
        [   0,    0,    0,    0,    6],
        [   0,    0,    0,    0,    2]])
mask:  tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1]], dtype=torch.uint8)
max_target_length:  9


In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [20]:
# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [21]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [22]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [23]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [24]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [25]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [39]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexFromSentences(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [64]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 10000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                           '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [30]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 10000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9705
Iteration: 2; Percent complete: 0.0%; Average loss: 8.8376
Iteration: 3; Percent complete: 0.0%; Average loss: 8.6437
Iteration: 4; Percent complete: 0.0%; Average loss: 8.3210
Iteration: 5; Percent complete: 0.1%; Average loss: 7.9873
Iteration: 6; Percent complete: 0.1%; Average loss: 7.4427
Iteration: 7; Percent complete: 0.1%; Average loss: 7.0909
Iteration: 8; Percent complete: 0.1%; Average loss: 6.9996
Iteration: 9; Percent complete: 0.1%; Average loss: 6.8118
Iteration: 10; Percent complete: 0.1%; Average loss: 6.3348
Iteration: 11; Percent complete: 0.1%; Average loss: 6.2822
Iteration: 12; Percent complete: 0.1%; Average loss: 5.7589
Iteration: 13; Percent complete: 0.1%; Average loss: 5.6285
Iteration: 14; Percent complete: 0.1%; Average loss: 5.8625
Iteration: 15; Percent complete: 0.1%; Average loss: 5.3982
Iteration: 16; Percent complete: 0.2%

Iteration: 136; Percent complete: 1.4%; Average loss: 4.1666
Iteration: 137; Percent complete: 1.4%; Average loss: 4.2859
Iteration: 138; Percent complete: 1.4%; Average loss: 4.2654
Iteration: 139; Percent complete: 1.4%; Average loss: 4.4399
Iteration: 140; Percent complete: 1.4%; Average loss: 4.4094
Iteration: 141; Percent complete: 1.4%; Average loss: 4.4648
Iteration: 142; Percent complete: 1.4%; Average loss: 4.3355
Iteration: 143; Percent complete: 1.4%; Average loss: 4.3257
Iteration: 144; Percent complete: 1.4%; Average loss: 4.2382
Iteration: 145; Percent complete: 1.5%; Average loss: 4.6411
Iteration: 146; Percent complete: 1.5%; Average loss: 4.4451
Iteration: 147; Percent complete: 1.5%; Average loss: 3.9729
Iteration: 148; Percent complete: 1.5%; Average loss: 3.9793
Iteration: 149; Percent complete: 1.5%; Average loss: 4.3251
Iteration: 150; Percent complete: 1.5%; Average loss: 4.2706
Iteration: 151; Percent complete: 1.5%; Average loss: 4.1433
Iteration: 152; Percent 

Iteration: 271; Percent complete: 2.7%; Average loss: 3.7287
Iteration: 272; Percent complete: 2.7%; Average loss: 3.9447
Iteration: 273; Percent complete: 2.7%; Average loss: 3.8016
Iteration: 274; Percent complete: 2.7%; Average loss: 4.0130
Iteration: 275; Percent complete: 2.8%; Average loss: 3.9350
Iteration: 276; Percent complete: 2.8%; Average loss: 4.0336
Iteration: 277; Percent complete: 2.8%; Average loss: 4.1680
Iteration: 278; Percent complete: 2.8%; Average loss: 4.0364
Iteration: 279; Percent complete: 2.8%; Average loss: 3.7453
Iteration: 280; Percent complete: 2.8%; Average loss: 3.8400
Iteration: 281; Percent complete: 2.8%; Average loss: 4.2319
Iteration: 282; Percent complete: 2.8%; Average loss: 3.5816
Iteration: 283; Percent complete: 2.8%; Average loss: 3.7711
Iteration: 284; Percent complete: 2.8%; Average loss: 4.1230
Iteration: 285; Percent complete: 2.9%; Average loss: 4.1973
Iteration: 286; Percent complete: 2.9%; Average loss: 4.2242
Iteration: 287; Percent 

Iteration: 406; Percent complete: 4.1%; Average loss: 3.6949
Iteration: 407; Percent complete: 4.1%; Average loss: 3.8080
Iteration: 408; Percent complete: 4.1%; Average loss: 3.5599
Iteration: 409; Percent complete: 4.1%; Average loss: 3.8371
Iteration: 410; Percent complete: 4.1%; Average loss: 3.6709
Iteration: 411; Percent complete: 4.1%; Average loss: 3.7115
Iteration: 412; Percent complete: 4.1%; Average loss: 3.7260
Iteration: 413; Percent complete: 4.1%; Average loss: 3.6634
Iteration: 414; Percent complete: 4.1%; Average loss: 3.9268
Iteration: 415; Percent complete: 4.2%; Average loss: 3.8404
Iteration: 416; Percent complete: 4.2%; Average loss: 3.5651
Iteration: 417; Percent complete: 4.2%; Average loss: 3.7327
Iteration: 418; Percent complete: 4.2%; Average loss: 3.7256
Iteration: 419; Percent complete: 4.2%; Average loss: 4.0311
Iteration: 420; Percent complete: 4.2%; Average loss: 3.6907
Iteration: 421; Percent complete: 4.2%; Average loss: 3.5761
Iteration: 422; Percent 

Iteration: 541; Percent complete: 5.4%; Average loss: 3.3227
Iteration: 542; Percent complete: 5.4%; Average loss: 3.7653
Iteration: 543; Percent complete: 5.4%; Average loss: 3.5345
Iteration: 544; Percent complete: 5.4%; Average loss: 3.6536
Iteration: 545; Percent complete: 5.5%; Average loss: 3.9330
Iteration: 546; Percent complete: 5.5%; Average loss: 3.7526
Iteration: 547; Percent complete: 5.5%; Average loss: 3.6889
Iteration: 548; Percent complete: 5.5%; Average loss: 3.6121
Iteration: 549; Percent complete: 5.5%; Average loss: 3.6045
Iteration: 550; Percent complete: 5.5%; Average loss: 3.6273
Iteration: 551; Percent complete: 5.5%; Average loss: 3.6848
Iteration: 552; Percent complete: 5.5%; Average loss: 3.7597
Iteration: 553; Percent complete: 5.5%; Average loss: 3.5031
Iteration: 554; Percent complete: 5.5%; Average loss: 3.4143
Iteration: 555; Percent complete: 5.5%; Average loss: 3.6390
Iteration: 556; Percent complete: 5.6%; Average loss: 3.3500
Iteration: 557; Percent 

...

Iteration: 9462; Percent complete: 94.6%; Average loss: 1.3172
Iteration: 9463; Percent complete: 94.6%; Average loss: 1.3364
Iteration: 9464; Percent complete: 94.6%; Average loss: 1.3918
Iteration: 9465; Percent complete: 94.7%; Average loss: 1.4007
Iteration: 9466; Percent complete: 94.7%; Average loss: 1.5432
Iteration: 9467; Percent complete: 94.7%; Average loss: 1.4098
Iteration: 9468; Percent complete: 94.7%; Average loss: 1.2959
Iteration: 9469; Percent complete: 94.7%; Average loss: 1.4479
Iteration: 9470; Percent complete: 94.7%; Average loss: 1.2681
Iteration: 9471; Percent complete: 94.7%; Average loss: 1.4590
Iteration: 9472; Percent complete: 94.7%; Average loss: 1.5359
Iteration: 9473; Percent complete: 94.7%; Average loss: 1.3215
Iteration: 9474; Percent complete: 94.7%; Average loss: 1.2629
Iteration: 9475; Percent complete: 94.8%; Average loss: 1.2457
Iteration: 9476; Percent complete: 94.8%; Average loss: 1.5154
Iteration: 9477; Percent complete: 94.8%; Average loss:

Iteration: 9593; Percent complete: 95.9%; Average loss: 1.2478
Iteration: 9594; Percent complete: 95.9%; Average loss: 1.3906
Iteration: 9595; Percent complete: 96.0%; Average loss: 1.3501
Iteration: 9596; Percent complete: 96.0%; Average loss: 1.1965
Iteration: 9597; Percent complete: 96.0%; Average loss: 1.3553
Iteration: 9598; Percent complete: 96.0%; Average loss: 1.3758
Iteration: 9599; Percent complete: 96.0%; Average loss: 1.4866
Iteration: 9600; Percent complete: 96.0%; Average loss: 1.4131
Iteration: 9601; Percent complete: 96.0%; Average loss: 1.2364
Iteration: 9602; Percent complete: 96.0%; Average loss: 1.2763
Iteration: 9603; Percent complete: 96.0%; Average loss: 1.3412
Iteration: 9604; Percent complete: 96.0%; Average loss: 1.2617
Iteration: 9605; Percent complete: 96.0%; Average loss: 1.3664
Iteration: 9606; Percent complete: 96.1%; Average loss: 1.1811
Iteration: 9607; Percent complete: 96.1%; Average loss: 1.3398
Iteration: 9608; Percent complete: 96.1%; Average loss:

Iteration: 9724; Percent complete: 97.2%; Average loss: 1.2840
Iteration: 9725; Percent complete: 97.2%; Average loss: 1.2361
Iteration: 9726; Percent complete: 97.3%; Average loss: 1.2747
Iteration: 9727; Percent complete: 97.3%; Average loss: 1.2402
Iteration: 9728; Percent complete: 97.3%; Average loss: 1.3762
Iteration: 9729; Percent complete: 97.3%; Average loss: 1.2073
Iteration: 9730; Percent complete: 97.3%; Average loss: 1.0910
Iteration: 9731; Percent complete: 97.3%; Average loss: 1.4317
Iteration: 9732; Percent complete: 97.3%; Average loss: 1.2627
Iteration: 9733; Percent complete: 97.3%; Average loss: 1.3298
Iteration: 9734; Percent complete: 97.3%; Average loss: 1.3091
Iteration: 9735; Percent complete: 97.4%; Average loss: 1.4469
Iteration: 9736; Percent complete: 97.4%; Average loss: 1.3215
Iteration: 9737; Percent complete: 97.4%; Average loss: 1.2549
Iteration: 9738; Percent complete: 97.4%; Average loss: 1.3352
Iteration: 9739; Percent complete: 97.4%; Average loss:

Iteration: 9855; Percent complete: 98.6%; Average loss: 1.2661
Iteration: 9856; Percent complete: 98.6%; Average loss: 1.2562
Iteration: 9857; Percent complete: 98.6%; Average loss: 1.4440
Iteration: 9858; Percent complete: 98.6%; Average loss: 1.2343
Iteration: 9859; Percent complete: 98.6%; Average loss: 1.3247
Iteration: 9860; Percent complete: 98.6%; Average loss: 1.2620
Iteration: 9861; Percent complete: 98.6%; Average loss: 1.3650
Iteration: 9862; Percent complete: 98.6%; Average loss: 1.2027
Iteration: 9863; Percent complete: 98.6%; Average loss: 1.2616
Iteration: 9864; Percent complete: 98.6%; Average loss: 1.2288
Iteration: 9865; Percent complete: 98.7%; Average loss: 1.3338
Iteration: 9866; Percent complete: 98.7%; Average loss: 1.3088
Iteration: 9867; Percent complete: 98.7%; Average loss: 1.3182
Iteration: 9868; Percent complete: 98.7%; Average loss: 1.2574
Iteration: 9869; Percent complete: 98.7%; Average loss: 1.3908
Iteration: 9870; Percent complete: 98.7%; Average loss:

Iteration: 9986; Percent complete: 99.9%; Average loss: 1.2167
Iteration: 9987; Percent complete: 99.9%; Average loss: 1.2285
Iteration: 9988; Percent complete: 99.9%; Average loss: 1.3267
Iteration: 9989; Percent complete: 99.9%; Average loss: 1.1741
Iteration: 9990; Percent complete: 99.9%; Average loss: 1.3234
Iteration: 9991; Percent complete: 99.9%; Average loss: 1.4348
Iteration: 9992; Percent complete: 99.9%; Average loss: 1.3286
Iteration: 9993; Percent complete: 99.9%; Average loss: 1.2165
Iteration: 9994; Percent complete: 99.9%; Average loss: 1.3400
Iteration: 9995; Percent complete: 100.0%; Average loss: 1.2758
Iteration: 9996; Percent complete: 100.0%; Average loss: 1.4078
Iteration: 9997; Percent complete: 100.0%; Average loss: 1.1871
Iteration: 9998; Percent complete: 100.0%; Average loss: 1.3412
Iteration: 9999; Percent complete: 100.0%; Average loss: 1.2991
Iteration: 10000; Percent complete: 100.0%; Average loss: 1.1549


In [65]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

evaluateInput(encoder, decoder, searcher, voc)

> hey there
Bot: hey . . . .
> what is your name
Bot: my name is ben . . .
> hey ben
Bot: hello . . .
> i love you
Bot: i love you too . !
> will you marry me?
Bot: bobo take your cart down . .
> sorry what does that mean?
Bot: don t ask . . .
> hmm okay
Bot: you re gonna kill my father .
> no
Bot: you re sure ? . .
> yes
Bot: i m glad you re fine .
> same
Bot: i ll tell you something . .
> tell me
Bot: tell them . .
> yes
Bot: i m glad you re fine .
> fuck yourself
Bot: i m scared . !
> try it
Bot: i ve got to go . you .
> no
Bot: you re sure ? . .
> i will stay
Bot: goodbye .
> goodbye
Bot: goodbye . . .


KeyboardInterrupt: 